In [13]:
import io
import pandas as pd
import requests
import csv
import re

In [45]:
FMP_API_ENDPOINT="https://financialmodelingprep.com/api"
FMP_API_KEY="6b5ead8d3c6bceb25d50bc6237dc8543"

In [46]:
def process_fmp_financial_statements(file_path):
    raw_df = pd.read_csv(file_path)
    raw_df.rename({"Unnamed: 1": "metric"}, axis=1, inplace=True)
    df = raw_df[~raw_df['metric'].isna()]
    df = df.drop("date", axis=1)
    df = df.rename({"metric": "date"}, axis=1)
    df = df.set_index("date").transpose()
    df.index = df.index.map(lambda s: re.sub(r"_Q[1-4]", "", s))
    df = df.sort_index(ascending=False)
    return df


In [63]:
def pull_fmp_financial_statements(stock_symbols, output_dir):
    statment_types = ["income-statement", "balance-sheet-statement", "cash-flow-statement"]
    session = requests.Session()
    for raw_sym in stock_symbols:
        sym = raw_sym.replace("/", "-")
        merged_file_name = f"{output_dir}/{sym}-all.csv"
        merged_df = None
        for stmt_type in statment_types:
            statement_url = f"{FMP_API_ENDPOINT}/v3/{stmt_type}/{sym}"
            resp = session.get(url = statement_url, params={
                    "apikey": FMP_API_KEY,
                    "period": "quarter",
                    "limit": 1,
                    "datatype": "csv"
                })
            if resp.status_code != 200:
                raise RuntimeError(f"http status is {resp.status_code}")
            file_name = f"{output_dir}/{sym}-{stmt_type}"
            raw_file_path = f"{file_name}-raw.csv"
            processed_file_path = f"{file_name}.csv"
            with open(raw_file_path, 'w', encoding='utf-8') as f:
                writer = csv.writer(f)
                reader = csv.reader(resp.content.decode('utf-8').splitlines())
                for row in reader:
                    if len(row) >= 3 and len(row[2]) > 0:
                        writer.writerow(row)
            df = process_fmp_financial_statements(raw_file_path)
            # df.to_csv(processed_file_path)
            if merged_df is not None:
                merged_df = merged_df.join(other=df, how="outer", rsuffix=f"_{stmt_type}")
            else:
                merged_df = df
        merged_df = merged_df.sort_index(ascending=False)
        merged_df.to_csv(merged_file_name, index=True)

In [64]:
output_dir="../../output"
pull_fmp_financial_statements(["AAPL"], output_dir)

# process_fmp_financial_statements(file_path="../../output/AAPL-balance-sheet-statement-raw.csv")

In [66]:
df = pd.read_csv("../../output/AAPL-all.csv", index_col=0)
df

,revenue,costOfRevenue,grossProfit,grossProfitRatio,ResearchAndDevelopmentExpenses,GeneralAndAdministrativeExpenses,SellingAndMarketingExpenses,otherExpenses,operatingExpenses,costAndExpenses,...,dividendsPaid,otherFinancingActivites,netCashUsedProvidedByFinancingActivities,effectOfForexChangesOnCash,netChangeInCash,cashAtEndOfPeriod,cashAtBeginningOfPeriod,operatingCashFlow,capitalExpenditure,freeCashFlow
2022-12-31,1.171540e+11,6.682200e+10,5.033200e+10,0.429623,7.709000e+09,0.0,0.0,0.0,1.431600e+10,8.113800e+10,...,-3.768000e+09,-2.705000e+09,-3.556300e+10,0.0,-3.003000e+09,2.197400e+10,2.497700e+10,3.400500e+10,-3.787000e+09,3.021800e+10
2022-09-24,9.014600e+10,5.205100e+10,3.809500e+10,0.422592,6.761000e+09,0.0,0.0,0.0,1.320100e+10,6.525200e+10,...,-3.703000e+09,4.130000e+09,-2.679400e+10,0.0,-3.884000e+09,2.497700e+10,2.886100e+10,2.412700e+10,-3.289000e+09,2.083800e+10
2022-06-25,8.295900e+10,4.707400e+10,3.588500e+10,0.432563,6.797000e+09,0.0,0.0,0.0,1.280900e+10,5.988300e+10,...,-3.811000e+09,1.231000e+09,-2.744500e+10,0.0,-3.190000e+08,2.886100e+10,2.918000e+10,2.289200e+10,-2.102000e+09,2.079000e+10
2022-03-26,9.727800e+10,5.471900e+10,4.255900e+10,0.437499,6.387000e+09,0.0,0.0,0.0,1.258000e+10,6.729900e+10,...,-3.595000e+09,1.625000e+09,-2.835100e+10,0.0,-9.450000e+09,2.918000e+10,3.863000e+10,2.816600e+10,-2.514000e+09,2.565200e+10
2021-12-25,1.239450e+11,6.970200e+10,5.424300e+10,0.437638,6.306000e+09,0.0,0.0,0.0,1.275500e+10,8.245700e+10,...,-3.732000e+09,-3.949000e+09,-2.815900e+10,0.0,2.701000e+09,3.863000e+10,3.592900e+10,4.696600e+10,-2.803000e+09,4.416300e+10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1986-09-30,5.108000e+08,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.0,0.0,0.0,0.000000e+00,0.000000e+00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1986-06-30,4.483000e+08,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.0,0.0,0.0,0.000000e+00,0.000000e+00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1986-03-31,4.089000e+08,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.0,0.0,0.0,0.000000e+00,0.000000e+00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1985-12-31,5.339000e+08,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.0,0.0,0.0,0.000000e+00,0.000000e+00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
